## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now() - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=6)

def format_date(date):
    day = date.day
    # Months 3-7 (March–July) should be full; all others abbreviated with a period
    if date.month in (3, 4, 5, 6, 7):
        month_str = date.strftime("%B")      # e.g. "March", "April", ...
    else:
        month_str = date.strftime("%b") + "."  # e.g. "Jan.", "Feb.", "Aug.", ...

    return f"{month_str} {day}"


# Format both dates
week_ending = format_date(today)
week_starting = format_date(one_week_ago)

print(week_starting, week_ending)

July 14 July 20


In [3]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,July-15-2025,Condo/Co-op,7935 East Dr #1202,North Bay Village,FL,33141.0,1000000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Bay-Village/79...,MARMLS,A11685123,N,Y,25.853021,-80.157273


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
df2 = df[(df['SOLD DATE'] >= one_week_ago) & (df['SOLD DATE'] < today)]

In [10]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-07-15    23
2025-07-16    23
2025-07-18    21
2025-07-17    16
2025-07-19     2
Name: count, dtype: int64

In [11]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [12]:
df_top_ten = df2.head(10)

In [13]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
102,PAST SALE,2025-07-15,Condo/Co-op,500 Alton Ph 4702,Miami Beach,FL,33139.0,17500000.0,4.0,6.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11694331,N,Y,25.775127,-80.141257
100,PAST SALE,2025-07-17,Condo/Co-op,2821 S Bayshore Dr Unit 9A,Miami,FL,33133.0,6900000.0,4.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2821-S-Bayshor...,MARMLS,A11694363,N,Y,25.728507,-80.238090
72,PAST SALE,2025-07-16,Condo/Co-op,300 Collins Ave Ph 1,Miami Beach,FL,33139.0,6300000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/300-Coll...,MARMLS,A11717273,N,Y,25.772635,-80.133689
41,PAST SALE,2025-07-15,Condo/Co-op,17001 Collins Ave Unit 4305 Duplex,Sunny Isles Beach,FL,33160.0,4200000.0,4.0,6.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11781403,N,Y,25.933587,-80.120917
104,PAST SALE,2025-07-15,Condo/Co-op,9501 Collins Ave Unit TH1,Surfside,FL,33154.0,4050000.0,4.0,5.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9501-Collin...,MARMLS,A11676507,N,Y,25.885460,-80.122050
88,PAST SALE,2025-07-18,Condo/Co-op,2678 Tigertail Ave #2001,Miami,FL,33133.0,3700000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2678-Tigertail...,MARMLS,A11711120,N,Y,25.731339,-80.236374
2,PAST SALE,2025-07-18,Condo/Co-op,1643 Brickell Ave #2502,Miami,FL,33129.0,3000000.0,3.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1643-Brickell-...,MARMLS,A11808027,N,Y,25.755513,-80.195018
47,PAST SALE,2025-07-16,Condo/Co-op,1000 Brickell Plz Ph 5603,Miami,FL,33131.0,3000000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Brickell-...,MARMLS,A11790411,N,Y,25.764264,-80.192725
68,PAST SALE,2025-07-15,Condo/Co-op,4775 Collins Ave #601,Miami Beach,FL,33140.0,2225000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4775-Col...,MARMLS,A11753622,N,Y,25.822448,-80.121668
86,PAST SALE,2025-07-16,Condo/Co-op,45 SW 9th St Unit LPH 4504,Miami,FL,33130.0,1400000.0,3.0,4.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/45-SW-9th-St-3...,MARMLS,A11703040,N,Y,25.765628,-80.193804


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [14]:
def process_response(url):
    try:
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"
        querystring = {"url": url}
        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)
        time.sleep(0.2)
        response.raise_for_status()

        # If .json() returns None, fall back on empty dict
        data = response.json() or {}

        # Drill down with or {} at each step to avoid None.get errors
        panel_info = (data.get('data') or {}) \
            .get('mainHouseInfoPanelInfo') or {}
        main_info = panel_info.get('mainHouseInfo') or {}

        # Now safe to pull lists
        listing_agents = main_info.get('listingAgents') or []
        buying_agents  = main_info.get('buyingAgents')  or []

        # Defaults
        list_agent_name_1 = list_broker_name_1 = None
        list_agent_name_2 = list_broker_name_2 = None
        buy_agent_name_1  = buy_broker_name_1  = None
        buy_agent_name_2  = buy_broker_name_2  = None

        # Extract up to two listing agents
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}) \
                                             .get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}) \
                                             .get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')

        # Extract up to two buying agents
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}) \
                                            .get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}) \
                                            .get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')

        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }


In [15]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [16]:
response_df = pd.DataFrame(response_list)

In [17]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [18]:
df_top_ten = merged_df

## Current Week's Values

In [19]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

85


In [20]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$1,049,321


In [21]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$607


In [22]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$89,192,316


In [23]:
def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None
    
last_week = load_last_week_data_df()

In [24]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
# previous_week_condo_sales = input()
previous_week_condo_sales = last_week['Condo Sales'].iloc[0]
print(previous_week_condo_sales)
print('-----------')

print('Input Previous Week Condo Average Sales Price:')
# previous_week_average_sales_price = input()
previous_week_average_sales_price = last_week['Average Sale Price'].iloc[0]
print(previous_week_average_sales_price)
print('-----------')

print('Input Previous Week Condo Average PSF:')
# previous_week_average_psf = input()
previous_week_average_psf = last_week['Average PSF'].iloc[0]
print(previous_week_average_psf)
print('-----------')

print('Input Previous Week Condo Sales total (ex: 198_000_000)')
# previous_week_condo_sales_total = input()
previous_week_condo_sales_total = last_week['Last Week Total Dollar Volume'].iloc[0]
print(previous_week_condo_sales_total)


## Clean Data

Input county name:


 MiamiDade


Input Previous Week Condo Sales Total (number units sold):
75
-----------
Input Previous Week Condo Average Sales Price:
1133152
-----------
Input Previous Week Condo Average PSF:
648
-----------
Input Previous Week Condo Sales total (ex: 198_000_000)
84986400.0


In [25]:
last_week

,Condo Sales,Average Sale Price,Average PSF,Last Week Total Dollar Volume
0,75,1133152,648,84986400.0


In [26]:
last_week['Last Week Total Dollar Volume'].iloc[0]

84986400.0

In [28]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

# Create a dictionary of your current week data.
current_week_data = {
    "Condo Sales": current_week_condo_sales,
    "Average Sale Price": average_sale_price,
    "Average PSF": average_price_psf,
    "Last Week Total Dollar Volume": current_week_condo_sales_total
}

# Convert the dictionary into a DataFrame.
current_week_summary_df = pd.DataFrame([current_week_data])

# Save the DataFrame to a CSV file.
save_current_week_data_df(current_week_summary_df)

# Next week, load last week's data automatically.
last_week_df = load_last_week_data_df()
if last_week_df is not None:
    print("Last week's data:")
    print(last_week_df)
else:
    print("No previous data found.")

Last week's data:
   Condo Sales  Average Sale Price  Average PSF  Last Week Total Dollar Volume
0           85             1049321          607                     89192316.0


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [29]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [30]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [31]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [32]:
pd.set_option('display.max_columns',None)

In [33]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-07-15,Condo/Co-op,500 Alton Ph 4702,Miami Beach,FL,33139.0,17500000.0,4.0,6.5,FIVE PARK,5888.0,NaN,2024.0,NaN,2972.0,15120.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11694331,N,Y,25.775127,-80.141257,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Anca Mirescu,Douglas Elliman,Charlotte Maietto,Douglas Elliman,Christopher Wands,Douglas Elliman,Charlotte Maietto,Douglas Elliman,1,orange


In [34]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [35]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-07-15,Condo/Co-op,500 Alton Ph 4702,Miami Beach,FL,33139.0,17500000.0,4.0,6.5,FIVE PARK,5888.0,NaN,2024.0,NaN,2972.0,15120.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11694331,N,Y,25.775127,-80.141257,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Anca Mirescu,Douglas Elliman,Charlotte Maietto,Douglas Elliman,Christopher Wands,Douglas Elliman,Charlotte Maietto,Douglas Elliman,1,orange
1,PAST SALE,2025-07-17,Condo/Co-op,2821 S Bayshore Dr Unit 9A,Miami,FL,33133.0,6900000.0,4.0,4.5,TWO PARK GROVE CONDO,3233.0,NaN,2018.0,NaN,2134.0,6592.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2821-S-Bayshor...,MARMLS,A11694363,N,Y,25.728507,-80.238090,https://www.redfin.com/FL/Miami/2821-S-Bayshor...,Julian Cohen,Douglas Elliman,None,None,Allison Blumenthal,Brown Harris Stevens,None,None,2,blue
2,PAST SALE,2025-07-16,Condo/Co-op,300 Collins Ave Ph 1,Miami Beach,FL,33139.0,6300000.0,3.0,3.5,300 COLLINS CONDO,3143.0,NaN,2018.0,NaN,2004.0,8310.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/300-Coll...,MARMLS,A11717273,N,Y,25.772635,-80.133689,https://www.redfin.com/FL/Miami-Beach/300-Coll...,Darryl Fox,One Sotheby's International Re,Daniela Bonetti,One Sotheby's International Re,Marko Gojanovic,One Sotheby's International Re,Reid Heidenry,One Sotheby's International Re,3,blue
3,PAST SALE,2025-07-15,Condo/Co-op,17001 Collins Ave Unit 4305 Duplex,Sunny Isles Beach,FL,33160.0,4200000.0,4.0,6.5,JADE BEACH CONDO,4141.0,NaN,2008.0,NaN,1014.0,6678.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11781403,N,Y,25.933587,-80.120917,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Carolina Lopez,One Sotheby's International Realty,None,None,Carolina Lopez,One Sotheby's International Realty,None,None,4,blue
4,PAST SALE,2025-07-15,Condo/Co-op,9501 Collins Ave Unit TH1,Surfside,FL,33154.0,4050000.0,4.0,5.5,OCEAN SEVEN CONOMINIUMS,3606.0,NaN,2014.0,NaN,1123.0,2300.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9501-Collin...,MARMLS,A11676507,N,Y,25.885460,-80.122050,https://www.redfin.com/FL/Surfside/9501-Collin...,Fernando Rodriguez,Fortune Christie's International Real Estate,None,None,TEMP USER,MIAMI Association of REALTORS®,None,None,5,blue
5,PAST SALE,2025-07-18,Condo/Co-op,2678 Tigertail Ave #2001,Miami,FL,33133.0,3700000.0,3.0,3.5,Mister C Residences,1811.0,NaN,2024.0,NaN,2043.0,3830.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2678-Tigertail...,MARMLS,A11711120,N,Y,25.731339,-80.236374,https://www.redfin.com/FL/Miami/2678-Tigertail...,Alexandra Sierra,"Compass Florida, LLC",None,None,Alexandra Sierra,"Compass Florida, LLC",None,None,6,blue
6,PAST SALE,2025-07-18,Condo/Co-op,1643 Brickell Ave #2502,Miami,FL,33129.0,3000000.0,3.0,4.5,SANTA MARIA,3170.0,NaN,1997.0,NaN,946.0,4110.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1643-Brickell-...,MARMLS,A11808027,N,Y,25.755513,-80.195018,https://www.redfin.com/FL/Miami/1643-Brickell-...,Fabian Dominguez,Fortune Christie's Intl R.E.,Peter Liakouras,Fortune Christie's Intl R.E.,Fabian Dominguez,Fortune Christie's Intl R.E.,Peter Liakouras,Fortune Christie's Intl R.E.,7,blue
7,PAST SALE,2025-07-16,Condo/Co-op,1000 Brickell Plz Ph 5603,Miami,FL,33131.0,3000000.0,3.0,3.5,BRICKELL FLATIRON CONDO,2076.0,NaN,2019.0,NaN,1445.0,3025.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Brickell-...,MARMLS,A11790411,N,Y,25.764264,-80.192725,https://www.redfin.com/FL/Miami/1000-Brickell-...,Diego Valencia,"Compass Florida, LLC",None,None,Liz Hogan,"Compass Florida, LLC.",None,None,8,blue
8,P

In [36]:
%store -r map_box_api_key

In [37]:
import folium
from branca.element import Template, MacroElement

# ——— Your title & caption HTML ———
title_html = f'''
  <h3 align="center" style="font-size:16px"><b>Recent {county} County Condo Sales</b></h3>
'''
caption_html = f'''
  <p align="center" style="font-size:13px"><i>{week_starting} — {week_ending}</i></p>
'''

# ——— Base map ———
m = folium.Map(
    location=df[['LATITUDE','LONGITUDE']].mean().to_list(),
    zoom_start=10.5,
    tiles=None
)

# ——— Tile layers ———
folium.TileLayer('OpenStreetMap', name='OSM', control=False).add_to(m)
folium.TileLayer(
    tiles=(
        "https://api.mapbox.com/styles/v1/"
        "mapbox/streets-v11/tiles/256/{z}/{x}/{y}@2x"
        f"?access_token={map_box_api_key}"
    ),
    attr="Mapbox", name="Streets", control=False
).add_to(m)

# ——— Add your markers ———
for _, row in df_top_ten.iterrows():
    lat, lon = row['LATITUDE'], row['LONGITUDE']
    popup = folium.Popup(popup_html(row), max_width=400)

    if row['COLOR'] == 'blue':
        # Regular sale: blue circle
        folium.CircleMarker(
            location=(lat, lon),
            radius=7,
            fill=True,
            fill_color='blue',
            fill_opacity=0.5,
            color=None,
            popup=popup
        ).add_to(m)
    else:
        # Top sale: gold star
        folium.Marker(
            location=(lat, lon),
            popup=popup,
            icon=folium.DivIcon(
                html="""
                <div style="font-size:24px; line-height:24px; color:gold;">
                  &#9733;
                </div>
                """
            )
        ).add_to(m)

# ——— Layer control, title & caption ———
folium.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

# ——— Optional legend via branca ———
legend_html = """
{% macro html(this, kwargs) %}
  <div style="
      position: fixed;
      bottom: 50px; left: 50px;
      background: white; padding: 8px;
      border: 2px solid grey;
      font-size: 12px;
      z-index: 9999;
  ">
    <div style="display:inline-block; font-size:18px; color:gold; margin-right:6px;">&#9733;</div> Top Sale<br>
    <i style="background: blue; width: 10px; height: 10px;
       display: inline-block; margin-right: 6px;"></i> All other sales
  </div>
{% endmacro %}
"""
legend = MacroElement()
legend._template = Template(legend_html)
m.get_root().add_child(legend)

# ——— Display ———
m


In [38]:
m.save('index.html')

## Data snagger

In [39]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [40]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-07-15,Condo/Co-op,500 Alton Ph 4702,Miami Beach,FL,33139.0,17500000.0,4.0,6.5,FIVE PARK,5888.0,NaN,2024.0,NaN,2972.0,15120.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11694331,N,Y,25.775127,-80.141257,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Anca Mirescu,Douglas Elliman,Charlotte Maietto,Douglas Elliman,Christopher Wands,Douglas Elliman,Charlotte Maietto,Douglas Elliman,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [41]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiDade_County_condo_sales_week_ending_07212025


# CREATE TEMPLATE 

In [42]:
muni_set = set(df_top_ten['CITY'])

In [43]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [44]:
df_top_ten.reset_index(inplace=True,drop=True)

In [45]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [46]:
top_sale

'FIVE PARK at 500 Alton Ph 4702 in Miami Beach'

In [47]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [48]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [49]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-07-15,Condo/Co-op,500 Alton Ph 4702,Miami Beach,FL,33139.0,17500000.0,4.0,6.5,FIVE PARK,5888.0,NaN,2024.0,NaN,2972.0,15120.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11694331,N,Y,25.775127,-80.141257,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Anca Mirescu,Douglas Elliman,Charlotte Maietto,Douglas Elliman,Christopher Wands,Douglas Elliman,Charlotte Maietto,Douglas Elliman,1,orange,FIVE PARK at 500 Alton Ph 4702 in Miami Beach
1,PAST SALE,2025-07-17,Condo/Co-op,2821 S Bayshore Dr Unit 9A,Miami,FL,33133.0,6900000.0,4.0,4.5,TWO PARK GROVE CONDO,3233.0,NaN,2018.0,NaN,2134.0,6592.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2821-S-Bayshor...,MARMLS,A11694363,N,Y,25.728507,-80.238090,https://www.redfin.com/FL/Miami/2821-S-Bayshor...,Julian Cohen,Douglas Elliman,None,None,Allison Blumenthal,Brown Harris Stevens,None,None,2,blue,TWO PARK GROVE CONDO at 2821 S Bayshore Dr Uni...
2,PAST SALE,2025-07-16,Condo/Co-op,300 Collins Ave Ph 1,Miami Beach,FL,33139.0,6300000.0,3.0,3.5,300 COLLINS CONDO,3143.0,NaN,2018.0,NaN,2004.0,8310.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/300-Coll...,MARMLS,A11717273,N,Y,25.772635,-80.133689,https://www.redfin.com/FL/Miami-Beach/300-Coll...,Darryl Fox,One Sotheby's International Re,Daniela Bonetti,One Sotheby's International Re,Marko Gojanovic,One Sotheby's International Re,Reid Heidenry,One Sotheby's International Re,3,blue,300 COLLINS CONDO at 300 Collins Ave Ph 1 in M...
3,PAST SALE,2025-07-15,Condo/Co-op,17001 Collins Ave Unit 4305 Duplex,Sunny Isles Beach,FL,33160.0,4200000.0,4.0,6.5,JADE BEACH CONDO,4141.0,NaN,2008.0,NaN,1014.0,6678.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11781403,N,Y,25.933587,-80.120917,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Carolina Lopez,One Sotheby's International Realty,None,None,Carolina Lopez,One Sotheby's International Realty,None,None,4,blue,JADE BEACH CONDO at 17001 Collins Ave Unit 430...
4,PAST SALE,2025-07-15,Condo/Co-op,9501 Collins Ave Unit TH1,Surfside,FL,33154.0,4050000.0,4.0,5.5,OCEAN SEVEN CONOMINIUMS,3606.0,NaN,2014.0,NaN,1123.0,2300.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9501-Collin...,MARMLS,A11676507,N,Y,25.885460,-80.122050,https://www.redfin.com/FL/Surfside/9501-Collin...,Fernando Rodriguez,Fortune Christie's International Real Estate,None,None,TEMP USER,MIAMI Association of REALTORS®,None,None,5,blue,OCEAN SEVEN CONOMINIUMS at 9501 Collins Ave Un...
5,PAST SALE,2025-07-18,Condo/Co-op,2678 Tigertail Ave #2001,Miami,FL,33133.0,3700000.0,3.0,3.5,Mister C Residences,1811.0,NaN,2024.0,NaN,2043.0,3830.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2678-Tigertail...,MARMLS,A11711120,N,Y,25.731339,-80.236374,https://www.redfin.com/FL/Miami/2678-Tigertail...,Alexandra Sierra,"Compass Florida, LLC",None,None,Alexandra Sierra,"Compass Florida, LLC",None,None,6,blue,Mister C Residences at 2678 Tigertail Ave #200...
6,PAST SALE,2025-07-18,Condo/Co-op,1643 Brickell Ave #2502,Miami,FL,33129.0,3000000.0,3.0,4.5,SANTA MARIA,3170.0,NaN,1997.0,NaN,946.0,4110.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1643-Brickell-...,MARMLS,A11808027,N,Y,25.755513,-80.195018,https://www.redfin.com/FL/Miami/1643-Brickell-...,Fabian Dominguez,Fortune Christie's Intl R.E.,Peter Liakouras,Fortune Christie's Intl R.E.,Fabian Dominguez,Fortune Christie's Intl R.E.,Peter Liakouras,Fortune Christie's Intl R.E.,7,blue,SANTA MARIA at 1643 Brickell Ave #2502 in Miami
7,PAST SALE,2025-07-16,Condo/Co-op,1000 Brickell Plz Ph 560

In [50]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: FIVE PARK closing tops MiamiDade weekly condo sales 
DEK: Top 10 sales ranged from $1,400,000 to $17,500,000
FEATURED HED:
SEO HED: MiamiDade County Weekly Condo Report 
SEO DESCRIPTION: MiamiDade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #MiamiDade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, MiamiDade, weekly condo sales, Miami Beach, Miami, Sunny Isles Beach, Surfside, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

MiamiDade County’s NEWS PEG HERE.

Brokers closed 85 condo sales totaling $89,192,316 million from July 14 to July 20. The previous week, brokers closed 75 condo sales totaling $84,986,400.

Last week’s units sold for an average of $1,049,321, lower than the $1,133,152 average price from the previous week. The av

In [51]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [52]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [53]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [54]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-07-15,Condo/Co-op,500 Alton Ph 4702,Miami Beach,FL,33139.0,17500000.0,4.0,6.5,FIVE PARK,5888.0,NaN,2024.0,NaN,2972.0,15120.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11694331,N,Y,25.775127,-80.141257,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Anca Mirescu,Douglas Elliman,Charlotte Maietto,Douglas Elliman,Christopher Wands,Douglas Elliman,Charlotte Maietto,Douglas Elliman,1,orange,FIVE PARK at 500 Alton Ph 4702 in Miami Beach
1,PAST SALE,2025-07-17,Condo/Co-op,2821 S Bayshore Dr Unit 9A,Miami,FL,33133.0,6900000.0,4.0,4.5,TWO PARK GROVE CONDO,3233.0,NaN,2018.0,NaN,2134.0,6592.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2821-S-Bayshor...,MARMLS,A11694363,N,Y,25.728507,-80.238090,https://www.redfin.com/FL/Miami/2821-S-Bayshor...,Julian Cohen,Douglas Elliman,None,None,Allison Blumenthal,Brown Harris Stevens,None,None,2,blue,TWO PARK GROVE CONDO at 2821 S Bayshore Dr Uni...
2,PAST SALE,2025-07-16,Condo/Co-op,300 Collins Ave Ph 1,Miami Beach,FL,33139.0,6300000.0,3.0,3.5,300 COLLINS CONDO,3143.0,NaN,2018.0,NaN,2004.0,8310.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/300-Coll...,MARMLS,A11717273,N,Y,25.772635,-80.133689,https://www.redfin.com/FL/Miami-Beach/300-Coll...,Darryl Fox,One Sotheby's International Re,Daniela Bonetti,One Sotheby's International Re,Marko Gojanovic,One Sotheby's International Re,Reid Heidenry,One Sotheby's International Re,3,blue,300 COLLINS CONDO at 300 Collins Ave Ph 1 in M...
3,PAST SALE,2025-07-15,Condo/Co-op,17001 Collins Ave Unit 4305 Duplex,Sunny Isles Beach,FL,33160.0,4200000.0,4.0,6.5,JADE BEACH CONDO,4141.0,NaN,2008.0,NaN,1014.0,6678.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11781403,N,Y,25.933587,-80.120917,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Carolina Lopez,One Sotheby's International Realty,None,None,Carolina Lopez,One Sotheby's International Realty,None,None,4,blue,JADE BEACH CONDO at 17001 Collins Ave Unit 430...
4,PAST SALE,2025-07-15,Condo/Co-op,9501 Collins Ave Unit TH1,Surfside,FL,33154.0,4050000.0,4.0,5.5,OCEAN SEVEN CONOMINIUMS,3606.0,NaN,2014.0,NaN,1123.0,2300.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9501-Collin...,MARMLS,A11676507,N,Y,25.885460,-80.122050,https://www.redfin.com/FL/Surfside/9501-Collin...,Fernando Rodriguez,Fortune Christie's International Real Estate,None,None,TEMP USER,MIAMI Association of REALTORS®,None,None,5,blue,OCEAN SEVEN CONOMINIUMS at 9501 Collins Ave Un...
5,PAST SALE,2025-07-18,Condo/Co-op,2678 Tigertail Ave #2001,Miami,FL,33133.0,3700000.0,3.0,3.5,Mister C Residences,1811.0,NaN,2024.0,NaN,2043.0,3830.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2678-Tigertail...,MARMLS,A11711120,N,Y,25.731339,-80.236374,https://www.redfin.com/FL/Miami/2678-Tigertail...,Alexandra Sierra,"Compass Florida, LLC",None,None,Alexandra Sierra,"Compass Florida, LLC",None,None,6,blue,Mister C Residences at 2678 Tigertail Ave #200...
6,PAST SALE,2025-07-18,Condo/Co-op,1643 Brickell Ave #2502,Miami,FL,33129.0,3000000.0,3.0,4.5,SANTA MARIA,3170.0,NaN,1997.0,NaN,946.0,4110.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1643-Brickell-...,MARMLS,A11808027,N,Y,25.755513,-80.195018,https://www.redfin.com/FL/Miami/1643-Brickell-...,Fabian Dominguez,Fortune Christie's Intl R.E.,Peter Liakouras,Fortune Christie's Intl R.E.,Fabian Dominguez,Fortune Christie's Intl R.E.,Peter Liakouras,Fortune Christie's Intl R.E.,7,blue,SANTA MARIA at 1643 Brickell Ave #2502 in Miami
7,PAST SALE,2025-07-16,Condo/Co-op,1000 Brickell Plz Ph 560

In [55]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: FIVE PARK closing tops MiamiDade weekly condo sales 
DEK: Top 10 sales ranged from $1,400,000 to $17,500,000
FEATURED HED:
SEO HED: MiamiDade County Weekly Condo Report 
SEO DESCRIPTION: MiamiDade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #MiamiDade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, MiamiDade, weekly condo sales, Miami Beach, Miami, Sunny Isles Beach, Surfside, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

MiamiDade County’s NEWS PEG HERE.

Brokers closed 85 condo sales totaling $89,192,316 million from July 14 to July 20. The previous week, brokers closed 75 condo sales totaling $84,986,400.

Last week’s units sold for an average of $1,049,321, lower than the $1,133,152 average price from the previous week. The av

In [63]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Miami-Beach/500-Alton-Rd-33139/unit-4702/home/193632264


In [64]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Miami/2821-S-Bayshore-Dr-33133/unit-9A/home/79134094


In [65]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Miami/45-SW-9th-St-33130/unit-4504/home/79134782


In [66]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Miami-Beach/4775-Collins-Ave-33140/unit-601/home/43248278


In [67]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Miami-Beach/500-Alton-Rd-33139/unit-4702/home/193632264


In [68]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Miami/45-SW-9th-St-33130/unit-4504/home/79134782


In [69]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-07-15,Condo/Co-op,500 Alton Ph 4702,Miami Beach,FL,33139.0,17500000.0,4.0,6.5,FIVE PARK,5888.0,NaN,2024.0,NaN,2972.0,15120.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11694331,N,Y,25.775127,-80.141257,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Anca Mirescu,Douglas Elliman,Charlotte Maietto,Douglas Elliman,Christopher Wands,Douglas Elliman,Charlotte Maietto,Douglas Elliman,1,orange,FIVE PARK at 500 Alton Ph 4702 in Miami Beach
1,PAST SALE,2025-07-17,Condo/Co-op,2821 S Bayshore Dr Unit 9A,Miami,FL,33133.0,6900000.0,4.0,4.5,TWO PARK GROVE CONDO,3233.0,NaN,2018.0,NaN,2134.0,6592.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2821-S-Bayshor...,MARMLS,A11694363,N,Y,25.728507,-80.238090,https://www.redfin.com/FL/Miami/2821-S-Bayshor...,Julian Cohen,Douglas Elliman,None,None,Allison Blumenthal,Brown Harris Stevens,None,None,2,blue,TWO PARK GROVE CONDO at 2821 S Bayshore Dr Uni...
2,PAST SALE,2025-07-16,Condo/Co-op,300 Collins Ave Ph 1,Miami Beach,FL,33139.0,6300000.0,3.0,3.5,300 COLLINS CONDO,3143.0,NaN,2018.0,NaN,2004.0,8310.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/300-Coll...,MARMLS,A11717273,N,Y,25.772635,-80.133689,https://www.redfin.com/FL/Miami-Beach/300-Coll...,Darryl Fox,One Sotheby's International Re,Daniela Bonetti,One Sotheby's International Re,Marko Gojanovic,One Sotheby's International Re,Reid Heidenry,One Sotheby's International Re,3,blue,300 COLLINS CONDO at 300 Collins Ave Ph 1 in M...
3,PAST SALE,2025-07-15,Condo/Co-op,17001 Collins Ave Unit 4305 Duplex,Sunny Isles Beach,FL,33160.0,4200000.0,4.0,6.5,JADE BEACH CONDO,4141.0,NaN,2008.0,NaN,1014.0,6678.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11781403,N,Y,25.933587,-80.120917,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Carolina Lopez,One Sotheby's International Realty,None,None,Carolina Lopez,One Sotheby's International Realty,None,None,4,blue,JADE BEACH CONDO at 17001 Collins Ave Unit 430...
4,PAST SALE,2025-07-15,Condo/Co-op,9501 Collins Ave Unit TH1,Surfside,FL,33154.0,4050000.0,4.0,5.5,OCEAN SEVEN CONOMINIUMS,3606.0,NaN,2014.0,NaN,1123.0,2300.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9501-Collin...,MARMLS,A11676507,N,Y,25.885460,-80.122050,https://www.redfin.com/FL/Surfside/9501-Collin...,Fernando Rodriguez,Fortune Christie's International Real Estate,None,None,TEMP USER,MIAMI Association of REALTORS®,None,None,5,blue,OCEAN SEVEN CONOMINIUMS at 9501 Collins Ave Un...
5,PAST SALE,2025-07-18,Condo/Co-op,2678 Tigertail Ave #2001,Miami,FL,33133.0,3700000.0,3.0,3.5,Mister C Residences,1811.0,NaN,2024.0,NaN,2043.0,3830.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2678-Tigertail...,MARMLS,A11711120,N,Y,25.731339,-80.236374,https://www.redfin.com/FL/Miami/2678-Tigertail...,Alexandra Sierra,"Compass Florida, LLC",None,None,Alexandra Sierra,"Compass Florida, LLC",None,None,6,blue,Mister C Residences at 2678 Tigertail Ave #200...
6,PAST SALE,2025-07-18,Condo/Co-op,1643 Brickell Ave #2502,Miami,FL,33129.0,3000000.0,3.0,4.5,SANTA MARIA,3170.0,NaN,1997.0,NaN,946.0,4110.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1643-Brickell-...,MARMLS,A11808027,N,Y,25.755513,-80.195018,https://www.redfin.com/FL/Miami/1643-Brickell-...,Fabian Dominguez,Fortune Christie's Intl R.E.,Peter Liakouras,Fortune Christie's Intl R.E.,Fabian Dominguez,Fortune Christie's Intl R.E.,Peter Liakouras,Fortune Christie's Intl R.E.,7,blue,SANTA MARIA at 1643 Brickell Ave #2502 in Miami
7,PAST SALE,2025-07-16,Condo/Co-op,1000 Brickell Plz Ph 560

## Time on Market Calculator

In [85]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 11, 15) ## List (Earlier) date
date2 = datetime(2025, 7, 15) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

242


## Clean CSV for Datawrapper Chart

In [70]:
chart_df = df_top_ten

In [71]:
chart_df = chart_df.fillna(" ")

In [72]:
chart_df['LISTING AGENTS'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING AGENTS'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [73]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [74]:
chart_df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME', 'LISTING AGENTS',
       'BUYING AGENTS'],
      dtype='object')

In [75]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "BEDS", 
    "BATHS",
    "SOURCE", 
    "MLS#"
]

In [76]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [77]:
chart_df = chart_df.rename(columns={'$/SQUARE FEET':'PPSF'})

In [78]:
chart_df['PPSF'].astype(float)

0    2972.0
1    2134.0
2    2004.0
3    1014.0
4    1123.0
5    2043.0
6     946.0
7    1445.0
8    1107.0
9     813.0
Name: PPSF, dtype: float64

In [79]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['PPSF'] = chart_df['PPSF'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))

In [80]:
desired_order = [
    'SOLD DATE',
    'LOCATION',
    'ADDRESS',
    'PRICE',
    'SQUARE FEET',
    'PPSF',
    'YEAR BUILT',
    'LISTING AGENTS',
    'BUYING AGENTS'
]

chart_df = chart_df[desired_order]


In [81]:
chart_df

,SOLD DATE,LOCATION,ADDRESS,PRICE,SQUARE FEET,PPSF,YEAR BUILT,LISTING AGENTS,BUYING AGENTS
0,2025-07-15,FIVE PARK,500 Alton Ph 4702 in Miami Beach,"$17,500,000","5,888","$2,972",2024,Anca Mirescu Douglas Elliman Charlotte Maietto...,Christopher Wands Douglas Elliman Charlotte Ma...
1,2025-07-17,TWO PARK GROVE CONDO,2821 S Bayshore Dr Unit 9A in Miami,"$6,900,000","3,233","$2,134",2018,Julian Cohen Douglas Elliman,Allison Blumenthal Brown Harris Stevens
2,2025-07-16,300 COLLINS CONDO,300 Collins Ave Ph 1 in Miami Beach,"$6,300,000","3,143","$2,004",2018,Darryl Fox One Sotheby's International Re Dani...,Marko Gojanovic One Sotheby's International Re...
3,2025-07-15,JADE BEACH CONDO,17001 Collins Ave Unit 4305 Duplex in Sunny Is...,"$4,200,000","4,141","$1,014",2008,Carolina Lopez One Sotheby's International Rea...,Carolina Lopez One Sotheby's International Rea...
4,2025-07-15,OCEAN SEVEN CONOMINIUMS,9501 Collins Ave Unit TH1 in Surfside,"$4,050,000","3,606","$1,123",2014,Fernando Rodriguez Fortune Christie's Internat...,TEMP USER MIAMI Association of REALTORS®
5,2025-07-18,Mister C Residences,2678 Tigertail Ave #2001 in Miami,"$3,700,000","1,811","$2,043",2024,"Alexandra Sierra Compass Florida, LLC","Alexandra Sierra Compass Florida, LLC"
6,2025-07-18,SANTA MARIA,1643 Brickell Ave #2502 in Miami,"$3,000,000","3,170",$946,1997,Fabian Dominguez Fortune Christie's Intl R.E. ...,Fabian Dominguez Fortune Christie's Intl R.E. ...
7,2025-07-16,BRICKELL FLATIRON CONDO,1000 Brickell Plz Ph 5603 in Miami,"$3,000,000","2,076","$1,445",2019,"Diego Valencia Compass Florida, LLC","Liz Hogan Compass Florida, LLC."
8,2025-07-15,GREEN DIAMOND CONDO,4775 Collins Ave #601 in Miami Beach,"$2,225,000","2,010","$1,107",2000,Claudia Whitechurch Coldwell Banker Realty Nat...,"Ryan Greenwald Property Pro Partners, LLC."
9,2025-07-16,BRICKELL HEIGHTS EAST CON,45 SW 9th St Unit LPH 4504 in Miami,"$1,400,000","1,723",$813,2017,"Stepan Rudenko Compass Florida, LLC Michael Ma...",Michael Wiesenfeld The Corcoran Group


In [82]:
csv_date_string = today.strftime("%m_%d_%Y")

In [83]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [84]:
chart_df.columns

Index(['SOLD DATE', 'LOCATION', 'ADDRESS', 'PRICE', 'SQUARE FEET', 'PPSF',
       'YEAR BUILT', 'LISTING AGENTS', 'BUYING AGENTS'],
      dtype='object')